In [10]:
# # CSV to VTK

# import pandas as pd
# import vtk
# import os

# # 设置CSV文件所在的文件夹路径
# folder_path = r"C:\Users\14014\Downloads\000"

# # 遍历文件夹中的所有CSV文件
# for filename in os.listdir(folder_path):
#     if filename.endswith('.csv'):
#         file_path = os.path.join(folder_path, filename)

#         # 读取CSV文件
#         df = pd.read_csv(file_path)

#         # 创建一个vtkPoints对象存储点数据
#         points = vtk.vtkPoints()
#         for index, row in df.iterrows():
#             points.InsertNextPoint(row['x'], row['y'], row['z'])

#         # 创建一个vtkPolyLine对象，它表示一系列线段，其中所有点都按顺序连接
#         polyLine = vtk.vtkPolyLine()
#         polyLine.GetPointIds().SetNumberOfIds(len(df))
#         for i in range(len(df)):
#             polyLine.GetPointIds().SetId(i, i)

#         # 创建一个vtkCellArray来存储这些线段
#         cells = vtk.vtkCellArray()
#         cells.InsertNextCell(polyLine)

#         # 创建一个vtkPolyData对象
#         polyData = vtk.vtkPolyData()
#         polyData.SetPoints(points)
#         polyData.SetLines(cells)

#         # 为点添加额外的数据，如curvature和torsion
#         curvature = vtk.vtkDoubleArray()
#         curvature.SetName("curvature")
#         torsion = vtk.vtkDoubleArray()
#         torsion.SetName("torsion")
#         for index, row in df.iterrows():
#             curvature.InsertNextValue(row['curvature'])
#             torsion.InsertNextValue(row['torsion'])
#         polyData.GetPointData().AddArray(curvature)
#         polyData.GetPointData().AddArray(torsion)

#         # 将vtkPolyData写入VTK文件
#         writer = vtk.vtkPolyDataWriter()
#         writer.SetFileName(file_path.replace('.csv', '.vtk'))
#         writer.SetInputData(polyData)
#         writer.Write()


INFO: Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO: NumExpr defaulting to 8 threads.


In [6]:
# 【不动】头文件和函数定义
import sys
sys.path.append('D:\\bai_work_win\\AngioMorph\\src')

import numpy as np 
import glob 
import os
import vtk
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from AngioMorphPCA.preprocessing import parameterize_curve, calculate_3d_curve_abscissas,remove_high_freq_components
from AngioMorphPCA.io import Get_simple_vtk, makeVtkFile, mkdir
from scipy.integrate import simps
from AngioMorphPCA.GenerateDiffHemeo import generate_diff_homeomorphism
from AngioMorphPCA.srvf_rep import compute_srvf_func, reconstruct_curve_from_srvf
from AngioMorphPCA.L2distance import calculate_l2_distance
from AngioMorphPCA.compute_geometry import compute_curvature_and_torsion,build_curve_from_curvatures
from tqdm import tqdm
import matplotlib.pyplot as plt
from geomstats.learning.frechet_mean import FrechetMean
import geomstats.backend as gs
from geomstats.geometry.discrete_curves import (
    DiscreteCurvesStartingAtOrigin,
    SRVMetric,
    insert_zeros,
)

def output_vtk_file(shape, output_file):
    # 创建VTK点数据和折线连接信息
    points = vtk.vtkPoints()
    polyline = vtk.vtkPolyLine()
    polyline.GetPointIds().SetNumberOfIds(shape.shape[0])  # 设置点的数量

    # 插入平均形状的点，并添加点到折线
    for i, point in enumerate(shape):
        points.InsertNextPoint(point.tolist())
        polyline.GetPointIds().SetId(i, i)

    # 创建包含折线的vtkCellArray
    lines = vtk.vtkCellArray()
    lines.InsertNextCell(polyline)

    # 创建vtkPolyData对象来保存点和连接信息
    poly_data = vtk.vtkPolyData()
    poly_data.SetPoints(points)
    poly_data.SetLines(lines)

    # 写入VTK文件（使用旧式的VTK文件格式）
    writer = vtk.vtkPolyDataWriter()
    writer.SetFileName(output_file)
    writer.SetInputData(poly_data)
    writer.Write()

    print(f"形状的VTK文件已保存至: {output_file}")



def compute_centroid(curves):
    centroid = np.mean(curves, axis=0)
    return np.array(centroid)
def translate_to_centroid(curves):
    centroid = compute_centroid(curves)
    new_curves = []
    for i in range(len(curves)):
        new_curves.append(curves[i] - centroid)
    return np.array(new_curves)

def plot_curve(curve, ax=None, add_origin=True):
    if ax is None:
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection="3d")

    if add_origin:
        curve = insert_zeros(curve, axis=-2)

    ax.plot(*[curve[:, k] for k in range(3)])
    ax.scatter(*[curve[0, k] for k in range(3)])
    return ax

def plot_geodesic(geod_points, ax=None, add_origin=True):
    n_times = geod_points.shape[0]
    k_sampling_points = geod_points.shape[-2] + 1

    if ax is None:
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_subplot(111, projection="3d")

    if add_origin:
        geod_points = insert_zeros(geod_points, axis=-2)

    ax.plot(*[geod_points[0, :, k] for k in range(3)],  c="b", linewidth=2)
    ax.plot(*[geod_points[-1, :, k] for k in range(3)], c="r", linewidth=2)

    for i in range(1, n_times - 1):
        ax.plot(*[geod_points[i, :, k] for k in range(3)], c="k", linewidth=1)

    for j in range(k_sampling_points - 1):
        ax.plot(*[geod_points[:, j, k] for k in range(3)], c="k", linewidth=1)

    return ax


In [3]:
# 顺天堂data pre-processing

import glob
import os

files = glob.glob('uezono_data/*.vtk')

print(files)

resample_num = 120

curves = []

output_folder = "uezono_data_mirrored/"

for file_path in files:
    # 从文件路径中提取文件名（不含扩展名）
    casename = os.path.basename(file_path).split('.')[0]
    

    temp = Get_simple_vtk(file_path)
    temp = temp[::-1]

    output_file_path = os.path.join(output_folder, f'{casename}.vtk')

    output_vtk_file(temp, output_file_path)



['uezono_data\\sub01-left.vtk', 'uezono_data\\sub01-right.vtk', 'uezono_data\\sub02-left.vtk', 'uezono_data\\sub02-right.vtk', 'uezono_data\\sub03-left.vtk', 'uezono_data\\sub03-right.vtk']
形状的VTK文件已保存至: uezono_data_mirrored/sub01-left.vtk
形状的VTK文件已保存至: uezono_data_mirrored/sub01-right.vtk
形状的VTK文件已保存至: uezono_data_mirrored/sub02-left.vtk
形状的VTK文件已保存至: uezono_data_mirrored/sub02-right.vtk
形状的VTK文件已保存至: uezono_data_mirrored/sub03-left.vtk
形状的VTK文件已保存至: uezono_data_mirrored/sub03-right.vtk


In [13]:
# def slice_curve_by_indices(curve_array, start_index, end_index):
#     """根据指定的起始和结束索引截取曲线的一部分，假设输入是numpy数组"""
#     if start_index < 0 or end_index >= len(curve_array) or start_index > end_index:
#         raise ValueError("Invalid start or end indices")

#     # 截取指定索引范围的点
#     sliced_points = curve_array[start_index:end_index + 1]

#     # 将numpy数组转换为vtkPoints
#     new_points = vtk.vtkPoints()
#     new_polyline = vtk.vtkPolyLine()
#     point_ids = new_polyline.GetPointIds()

#     # 添加点到vtkPoints，并记录索引到vtkPolyLine
#     for i, pt in enumerate(sliced_points):
#         idx = new_points.InsertNextPoint(pt.tolist())  # numpy数组转列表
#         point_ids.InsertNextId(idx)

#     # 创建vtkPolyData对象
#     new_polydata = vtk.vtkPolyData()
#     new_polydata.SetPoints(new_points)

#     cells = vtk.vtkCellArray()
#     cells.InsertNextCell(new_polyline)
#     new_polydata.SetLines(cells)
#     return new_polydata

# def save_vtk_curve(curve, output_path):
#     """将截取的曲线保存为VTK文件"""
#     writer = vtk.vtkPolyDataWriter()
#     writer.SetFileName(output_path)
#     writer.SetInputData(curve)
#     writer.Write()

# input_file = 'uezono_data_mirrored/sub03-right.vtk'  # 替换为你的文件路径
# output_file = 'uezono_data_mirrored1/sub03-right.vtk'  # 替换为输出文件的路径
# start_index = 25  # 截取的起始索引
# end_index = 340   # 截取的结束索引

# original_curve = Get_simple_vtk(input_file)
# sliced_curve = slice_curve_by_indices(original_curve, start_index, end_index)

# save_vtk_curve(sliced_curve, output_file)

In [31]:
# 然后左右上下镜像

def up_down_mirror_plane(start_point, end_point):
    direction = end_point - start_point
    normal = direction / np.linalg.norm(direction) # 归一化方向向量
    return normal

def mirror_points(points, normal, point_on_plane):
    mirrored_points = []
    for p in points:
        projection = np.dot(p - point_on_plane, normal)
        mirrored_p = p - 2 * projection * normal
        mirrored_points.append(mirrored_p)
        #print(f"Original: {p}, Projection: {projection}, Mirrored: {mirrored_p}, Normal: {normal}, Point on Plane: {point_on_plane}")
    return np.array(mirrored_points)



In [32]:

# 左右镜像找不到参考，就随便选了一下
# Load standard curves
curve_R = Get_simple_vtk('brava_average_R.vtk')
curve_L = Get_simple_vtk('brava_average_L.vtk')

# Compute centroids
center_R = compute_centroid(curve_R)
center_L = compute_centroid(curve_L)

# Compute mirror plane normal
normal = (center_L - center_R) / np.linalg.norm(center_L - center_R)
mid_point = (center_R + center_L) / 2

In [36]:
files = glob.glob('uezono_data_mirrored/*.vtk')
print(files)

output_folder = "uezono_data_mirrored_output/"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


for file_path in files:
    casename = os.path.basename(file_path)
    curve = Get_simple_vtk(file_path)
    
    # Mirror each curve
    mirrored_curve = mirror_points(curve, normal, mid_point)
    
    # Save the mirrored curve
    output_file_path = os.path.join(output_folder, f'{casename}')
    output_vtk_file(mirrored_curve, output_file_path)  # Assuming no line data for simplicity

print("Mirroring complete.")

['uezono_data_mirrored\\sub01_L.vtk', 'uezono_data_mirrored\\sub01_R.vtk', 'uezono_data_mirrored\\sub02_L.vtk', 'uezono_data_mirrored\\sub02_R.vtk', 'uezono_data_mirrored\\sub03_L.vtk', 'uezono_data_mirrored\\sub03_R.vtk']
形状的VTK文件已保存至: uezono_data_mirrored_output/sub01_L.vtk
形状的VTK文件已保存至: uezono_data_mirrored_output/sub01_R.vtk
形状的VTK文件已保存至: uezono_data_mirrored_output/sub02_L.vtk
形状的VTK文件已保存至: uezono_data_mirrored_output/sub02_R.vtk
形状的VTK文件已保存至: uezono_data_mirrored_output/sub03_L.vtk
形状的VTK文件已保存至: uezono_data_mirrored_output/sub03_R.vtk
Mirroring complete.
